In [3]:
from geopy.geocoders import Nominatim

# Инициализация геокодера
geolocator = Nominatim(user_agent="geoapi")

try:
    location = geolocator.geocode("томск, Россия")
    print(f"Координаты центра: {location.latitude}, {location.longitude}")
except Exception as e:
    print(e)
# Полный ответ с деталями
print(location.raw) 

Координаты центра: 56.4887526, 84.9523434
{'place_id': 201214899, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 7261645, 'lat': '56.4887526', 'lon': '84.9523434', 'class': 'place', 'type': 'city', 'place_rank': 16, 'importance': 0.608877673585454, 'addresstype': 'city', 'name': 'Томск', 'display_name': 'Томск, городской округ Томск, Томская область, Сибирский федеральный округ, 634000, Россия', 'boundingbox': ['56.3793680', '56.5705856', '84.8275215', '85.1495558']}


In [ ]:
import os
import json

CACHE_FILE = "resource/geo_cache.json"`
geolocator = Nominatim(user_agent="geoapi", timeout=10)

# Загрузка кэша из файла
def load_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, 'r', encoding='utf-8') as f:
                return json.load(f)
        except:
            return {}
    return {}

['РјРѕСЃРєРІР° [10,10]\n', 'РЎР°РЅРєС‚ РџРµС‚РµСЂР±СѓСЂРі [18,18]\n', 'РўРѕРјСЃРє [90,90]']


---
---

In [ ]:
import json
import os
from geopy.geocoders import Nominatim
import pandas as pd

# Конфигурация
CACHE_FILE = "geo_cache.json"
geolocator = Nominatim(user_agent="geoapi", timeout=10)

# Загрузка кэша из файла
def load_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, 'r', encoding='utf-8') as f:
                return json.load(f)
        except:
            return {}
    return {}

# Сохранение кэша в файл
def save_cache(cache):
    with open(CACHE_FILE, 'w', encoding='utf-8') as f:
        json.dump(cache, f, ensure_ascii=False, indent=2)

# Получение координат с использованием кэша
def get_geopoints(city: str, cache: dict) -> tuple:
    if not city or pd.isna(city):
        return (None, None)
    
    # Проверяем кэш
    if city in cache:
        return tuple(cache[city])
    
    # Геокодирование если нет в кэше
    try:
        location = geolocator.geocode(f"{city}, Россия")
        if location:
            result = (location.latitude, location.longitude)
        else:
            result = (None, None)
    except Exception:
        result = (None, None)
    
    # Обновляем кэш
    cache[city] = result
    return result

# Основная функция обработки DataFrame
def process_geodata(df: pd.DataFrame) -> pd.DataFrame:
    cache = load_cache()
    cache_updated = False
    
    # Применяем только к строкам с отсутствующими геоданными
    mask = df['geo'].isna()
    
    for idx in df.index[mask]:
        city = df.at[idx, 'city']
        if pd.notna(city):
            # Получаем координаты (обновляем кэш внутри функции)
            new_geo = get_geopoints(city, cache)
            df.at[idx, 'geo'] = new_geo
            cache_updated = True
    
    # Сохраняем кэш если были изменения
    if cache_updated:
        save_cache(cache)
        
    return df

# Использование
full_df = process_geodata(full_df)

In [ ]:
{
  "Москва": [55.7504461, 37.6174943],
  "Санкт-Петербург": [59.938732, 30.316229],
  "Новосибирск": [55.008352, 82.935732],
  "Екатеринбург": [56.838002, 60.597295]
}